## IMPORTING LIBRARIES

In [20]:
import pandas as pd
import numpy as np

## READING DATA SET

In [21]:
Train = pd.read_excel('Data_Latih_BDC.xlsx')
stopwords = pd.read_csv('stopwords.csv')
stop = stopwords['stopwords'].to_list()

KeyError: 'stopwords'

In [ ]:
Train.head()

## DATA PRE-PROCESSING

In [ ]:
X = Train.drop('label', axis=1)

In [ ]:
X.head()

In [ ]:
Train.shape

In [ ]:
Train.isnull().sum()

In [ ]:
Train.head(10)

In [ ]:
x = Train['narasi']
y = Train['label']
y.shape

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import re

In [ ]:
from nltk.corpus import stopwords
factory = StemmerFactory()
stemmer = factory.create_stemmer()
corpus = []
for i in range(0, len(Train)):
    review = re.sub('r"\d+"', ' ', Train['narasi'][i])
    reivew = review.lower()
    review = review.split()
    
    review = [stemmer.stem(word) for word in review if not word in (stop)]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[70]

In [ ]:
voc_size = 5000

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
one_hot_rep = [one_hot(words, voc_size) for words in corpus]

In [ ]:
len(one_hot_rep[0])

In [ ]:
len(one_hot_rep[70])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentence_length = 75

embedded_docs = pad_sequences(one_hot_rep, padding = 'pre', maxlen = sentence_length)
print(embedded_docs)

## BUILDING MODELS

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1, activation='tanh'))
model.compile(loss = 'binary_crossentropy', optimizer = 'Adadelta', metrics = ['accuracy'])

In [ ]:
z = np.array(embedded_docs)
y = np.array(y)

In [ ]:
z.shape, y.shape

## Spiliting and Training

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(z, y, test_size=0.20 ,random_state=42)

In [ ]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs= 20, batch_size=20)

In [ ]:
Test = pd.read_excel('data/Data_Uji_BDC.xlsx')
Test_id = Test['ID']

In [ ]:
Test_id

In [ ]:
Test.head()

In [ ]:
Test = Test.drop(['tanggal','ID','judul','nama file gambar'], axis=1)

In [ ]:
Test.head()

In [ ]:
Test.isnull().sum()

In [ ]:
Test.shape

In [ ]:
from nltk.corpus import stopwords
factory = StemmerFactory()
stemmer = factory.create_stemmer()
corpus_test = []
for i in range(0, len(Test)):
    review = re.sub('r"\d+"', ' ', Test['narasi'][i])
    reivew = review.lower()
    review = review.split()
    
    review = [stemmer.stem(word) for word in review if not word in (stop)]
    review = ' '.join(review)
    corpus_test.append(review)

In [ ]:
corpus_test[30]

In [ ]:
one_hot_rep_Test = [one_hot(words, voc_size) for words in corpus_test]

In [ ]:
sentence_length = 50

embedded_docs_test = pad_sequences(one_hot_rep_Test, padding = 'pre', maxlen = sentence_length)
print(embedded_docs_test)

In [ ]:
x_test= np.array(embedded_docs_test)

In [ ]:
check = model.predict_classes(x_test)

In [ ]:
check

In [ ]:
check.shape

In [ ]:
Test.shape

## Submission File

In [ ]:
val = []
for i in check:
    val.append(i[0])

In [ ]:
submission = pd.DataFrame({'ID':Test_id, 'label':val})
submission.shape

In [ ]:
submission.head(50)

In [ ]:
submission.to_csv('Hasil.csv',index=False)